In [1]:
import sys
import time

import pickle as pkl
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np


from Extractor import Extractor
from scipy.sparse import coo_matrix,csr_matrix
#import tensorflow as tf
import torch
from utils import *
from model import *
from train import *

from explain import *
import time
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import to_networkx


np.set_printoptions(threshold=sys.maxsize)

/home/grads/q/qf31/anaconda3/envs/graph/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='./dataset/Mutagenicity', name='Mutagenicity', use_node_attr=True)
dataset.len()

4337

In [3]:
gcn_model = torch.load('/data/qf31/DEGREE/MUTAG_search_checkpoint_2/auc_0.9326872275822641', map_location=torch.device(device))

In [4]:
select = np.load('gt_index.npy')
edge_label_lists = np.load('edge_label_lists.npy', allow_pickle=True)
edge_lists = np.load('edge_lists.npy', allow_pickle = True)

In [5]:
kwargs = {}
kwargs['node_name_list'] = ['C', 'O', 'Cl', 'H', 'N', 'F', 'Br', 'S', 'P', 'I', 'Na', 'K', 'Li', 'Ca']

In [6]:
from utils import *
import torch
from CD_layers import *
import time
from torch_geometric.utils import to_networkx
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt

def find_edge(model, dataset, idx, class_idx = None, node_sort = None, topk = None, start_num = 2):
    if node_sort is None:
        node_sort, node_color = print_explain(dataset, model, idx, class_idx = class_idx, visible = False)
    if class_idx is None:
        class_idx = int(dataset[idx].y[0])
    data = get_data(dataset, idx)
    data = data.to(device)
    start_num = start_num
    
    rest = list(node_sort.copy())
    select = set()
    neighbor_nodes = set()
    select_node_progress = []
    edge_index = np.array(dataset[idx].edge_index)

    while len(select) < topk and len(rest) != 0:
        if len(neighbor_nodes) == 0:
            for _ in range(start_num):
                if len(rest) == 0:
                    pass
                select_node = rest[0]
                select.add(select_node)
                rest.remove(select_node)
                # add new neighbor edge
                temp_neighbor_idx = (np.where(edge_index[0] == select_node))[0]
                #print(temp_neighbor_idx)
                for n_idx in temp_neighbor_idx:
                    n = int(edge_index[1][n_idx])
                    if n not in select:
                        neighbor_nodes.add(n)
                try:
                    neighbor_nodes.remove(select_node)
                except:
                    pass
                #print('no neighbor')
                #print('select: ', select)
                #print('neighbor_nodes', neighbor_nodes)
            select_node_progress.append(select.copy())
            
        else:
            # prepare list for score
            check_list = []
            for node in neighbor_nodes:
                group = select.copy()
                group.add((int)(node))
                check_list.append(group)
            #print(check_list)
            class_score = get_score(model, data, mask_list = check_list.copy())
            group_scores = class_score[class_idx]['rel']
            group_rank = np.argsort(group_scores)[::-1]
            print('score: ', group_scores[group_rank[0]])
            #print(group_rank)
            select = check_list[group_rank[0]]
            #print(check_list)
            intersect = select.intersection(neighbor_nodes)
            #print(intersect)
            neighbor_nodes = neighbor_nodes - select
            #print(rest)
            for node in intersect:
                rest.remove(node)
                # add new neighbor edge
                temp_neighbor_idx = (np.where(edge_index[0] == node))[0]
                #print('temp_neighbor_idx', temp_neighbor_idx)
                for n_idx in temp_neighbor_idx:        
                    n = int(edge_index[1][n_idx])
                    if n not in select:
                        neighbor_nodes.add(n)
            select_node_progress.append(select.copy())
        size = max(edge_index[0]) + 1
        adj = np.zeros((size, size))
        edge_pred = []
        for i in range(edge_index.shape[1]):
            r = edge_index[0][i]
            c = edge_index[1][i]
            if r in select and c in select:
                adj[r][c] = 1
                adj[c][r] = 1
            
            if r < c:
                if r in select and c in select:
                    edge_pred.append(1)
                else:
                    edge_pred.append(0)

        adj = preprocess_adj(adj)
    return select,select_node_progress,adj,edge_pred

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def CD_explain(model, dataset, idx = 0, mask_node_list = None, node_range = None):
    """
    idx: idx of graph to explain
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    data = get_data(dataset, idx)
    data = data.to(device)


    mask_list = []
    # generate mask
    node_range = data.x.shape[0] if node_range is None else node_range


    for i in range(node_range):
        if mask_node_list is not None and i in mask_node_list:
            mask_list.append(mask_node_list)
        else:
            mask_list.append([i])
    # forward to explain according to mask list
    class_score = get_score(model, data, mask_list)
    return class_score
def get_score(model, data, mask_list):

    for i in range(len(mask_list)):
        mask_index = [0] * data.x.shape[0]
        for m in mask_list[i]:
            mask_index[m] = 1
        mask_list[i] = mask_index
    
    preds_list = []
    for mask_index in mask_list:
        model.zero_grad()
        preds = model.forward(data, CD_explain=True, mask_index = mask_index)
        #print(preds)
        #print(data.y)
        preds['rel'] = preds['rel'].T
        preds['irrel'] = preds['irrel'].T
        #print(preds['irrel'].shape[0])
        #print(dataset.num_classes)
        preds = CD_softmax(preds, index = torch.tensor([0] * preds['irrel'].shape[0]).to(device))
        preds_list.append(preds)
        # need to softmax??

    # rel cd score for each class
    class_score = {}
    for class_idx in range(preds['irrel'].shape[0]):
        class_score[class_idx] = {}
        class_score[class_idx]['rel'] = []
        class_score[class_idx]['irrel'] = []
        for preds in preds_list:
            class_score[class_idx]['rel'].append((float)((preds['rel'][class_idx][0].cpu().detach())))
            class_score[class_idx]['irrel'].append((float)((preds['irrel'][class_idx][0].cpu().detach())))
            #class_score[class_idx]['rel'].append((float)((preds['rel'][0][class_idx].cpu().detach())))
            #class_score[class_idx]['irrel'].append((float)((preds['irrel'][0][class_idx].cpu().detach())))

    return class_score

def print_explain(dataset, model, idx, class_idx = None, visible = True, figsize = None, node_range = None, **kwargs):
    
    class_idx = class_idx if class_idx is not None else (int)(dataset[idx].y[0])
    
    begin_time = time.time()
    class_score = CD_explain(model, dataset, idx, node_range)
    
    print('prediction score',(class_score[0]['rel'][0] + class_score[0]['irrel'][0]))
    elapsed = time.time() - begin_time
    node_colors = class_score[class_idx]['rel']
    size_ratio = 1.0 + (node_colors - np.min(node_colors)) / (np.max(node_colors) - np.min(node_colors))
    print('epoch time: ', elapsed)

    if visible:
        if figsize is not None:
            plt.figure(figsize=figsize)
        G = to_networkx(dataset[idx])

        pos = nx.kamada_kawai_layout(G)

        node_sizes = 600 * size_ratio
        node_colors = class_score[class_idx]['rel']


        cmap = plt.cm.YlGnBu

        #t = [0, 1, 2, 4, 5, 6, 8, 9, 10, 17]
        #for tt in t:
        #    node_colors[tt] = 0.4264645576477051
        nodes = nx.draw_networkx_nodes(
            G,
            pos,
            node_size=node_sizes,
            node_color=node_colors,
            cmap=cmap,
            edgecolors='blue',
            vmax = max(node_colors) * 1.2,
            #vmin = 0.1
        )
        edges = nx.draw_networkx_edges(
            G,
            pos,
            node_size=node_sizes,
            arrowstyle="-",
            arrowsize=10,
            width=2,
        )
        label_list = {}
        for i in range(dataset[idx].x.shape[0]):
            label_list[i] = str(i)
            if 'node_name_list' in kwargs:
                node_name_idx = int(np.argmax(dataset[idx].x[i]))
                label_list[i] += '' + kwargs['node_name_list'][node_name_idx]

        labels = nx.draw_networkx_labels(G, pos, label_list, font_size=12, font_color="black")
        plt.colorbar(nodes)

        ax = plt.gca()
        ax.set_axis_off()
        #plt.savefig('1')
        plt.show()

    return np.argsort(node_colors)[::-1], node_colors

In [7]:
def Edge_explain(model, dataset, edge_list, idx = 0, node_range = None):
    """
    idx: idx of graph to explain
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    data = get_data(dataset, idx)
    data = data.to(device)


    mask_list = []
    # generate mask
    node_range = data.x.shape[0] if node_range is None else node_range

    edge_index = data.edge_index
    
    mask_list = edge_list
    #for i in range(len(mask_list)):
    #    print(i, ' : ', mask_list[i])

    # forward to explain according to mask list
    class_score = get_score(model, data, mask_list)
    return class_score

In [8]:
acc_list = []
auc_list = []
node_num_list = []

load_model = gcn_model

for idx in range(4337):
    idx = int(select[idx])
    print('\nindex: ', idx)
    data = dataset[idx]

    print('label: ', data.y[0])

    edge_score = Edge_explain(load_model, dataset, edge_lists[idx], idx)
    edge_colors = []
    for i in range(len(edge_score[0]['rel'])):
        edge_colors.append( edge_score[0]['rel'][i] - edge_score[1]['rel'][i] )
    print('edge sort: ', np.argsort(edge_colors)[::-1])
    auc = roc_auc_score(edge_label_lists[idx],edge_colors)
    print('auc: ', auc)
    auc_list.append(auc)
    print('auc mean: ', np.mean(auc_list))



index:  10
label:  tensor(0)
edge sort:  [21 20 19 18 14 15  2  3 11 10  0  1  5  4  6  7 35 34 22 23 12 13  8  9
 41 40 25 24 38 39 46 47 28 29 17 16 49 48 42 45 36 37 43 44 55 54 51 50
 26 27 33 32 31 30 56 57 52 53]
auc:  1.0
auc mean:  1.0

index:  22
label:  tensor(0)
edge sort:  [26 27 28 29 32 33  0  1 20 21 18 19  4  5 12 13 39 38 44 45  8  9 22 23
 40 41 17 16 47 46 35 36  7 34 43 37 42  6 25 24 50 48 49 51 30 31 14 15
  3  2 11 10]
auc:  1.0
auc mean:  1.0

index:  25
label:  tensor(0)
edge sort:  [61 60 63 62  6  7 18 19 16 17 30 31 28 29 35 34  1  0  4  5 40 41 11 10
  2  3 39 38 53 52 51 50 65 64 66 20 21 22 67 23 69 68 37 36 47 46 45 44
 43 42 56 13 49 48 12 25 59 58 57 33 32 54 24 55 15  9  8 14 27 26]
auc:  1.0
auc mean:  1.0

index:  31
label:  tensor(0)
edge sort:  [ 4  5  2  3 44 45 42 43  0  1 12 13 10 11 15 14  8  9  7  6 30 31 41 40
 22 26 23 24 25 27 51 50 35 34 36 39 38 37 47 46 53 52 54 33 32 55 28 16
 17 18 19 48 49 20 21 29]
auc:  0.9230769230769231
auc mean

IndexError: index 1015 is out of bounds for axis 0 with size 1015